# Statistik zur Fachzugehörigkeit

In [43]:
import pandas as pd
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

@cached
def getParent(termId):
    return querySingleton("""
        select parent_id from term_taxonomy where id = %s;
    """ % termId)[0]

def getTermName(termId):
    return querySingleton("""
        select term.name from term_taxonomy
        join term on term.id = term_taxonomy.term_id
        where term_taxonomy.id = %s;
    """ % termId)[0]

@cached
def getSubject(termId):
    parent = getParent(termId)
    grandparent = getParent(parent)
    
    if (parent == 106081):
        return getTermName(termId)
    
    return getSubject(parent) if grandparent != None else getTermName(termId)


def getSubjects(userId):
    df = pd.read_sql("""
        select term_taxonomy_entity.term_taxonomy_id as termid
        from event_log join uuid on uuid.id = event_log.uuid_id
        join entity_revision on entity_revision.id = uuid.id
        join entity on entity.id = entity_revision.repository_id
        join term_taxonomy_entity on term_taxonomy_entity.entity_id  = entity.id
        where event_log.event_id = 5
            and event_log.instance_id = 1
            and uuid.discriminator = "entityRevision"
            and event_log.actor_id = %s;
    """ % userId, db)
    df["subject"] = df["termid"].map(getSubject)
    return df.groupby("subject").count().rename(columns={"termid": "edits"})
    
get_subjects(266)

,edits
subject,
Chemie,7
Community,10
Informatik,2
Latein,25
Mathe,3054
Physik,19


# Diagramm zur Fachzugehörigkeit

In [155]:
import math

from IPython.display import HTML, display

colors = ["#1E65A7", "#F1CD04", "#00743F", "#D87A00"]

def polarToCartesian(x, y, radius, angleInDegrees):
        angleInRadians = (angleInDegrees - 90) * math.pi / 180.0

        return ( x + (radius * math.cos(angleInRadians)),
                 y + (radius * math.sin(angleInRadians)))
    
def circle(x, y, radius, startAngle, endAngle, color="black"):
    start = polarToCartesian(x, y, radius, endAngle)
    end = polarToCartesian(x, y, radius, startAngle)

    largeArcFlag = "0" if endAngle - startAngle <= 180 else "1"

    d = " ".join(map(str, [
        "M", start[0], start[1], 
        "A", radius, radius, 0, largeArcFlag, 0, end[0], end[1]
    ]))


    return """
        <path d="%s" fill="none" stroke-linecap="butt"
              stroke="%s" stroke-width="%s" />
    """ % (d, color, 16)

def createSubjectChart(subjects):
    subjects = subjects.sort_values(by=["edits"], ascending=False)
    subjects["percentage"] = subjects["edits"] / subjects["edits"].sum() * 100
    subjects = subjects[subjects["percentage"] > 1.5]
    subjects["percentage"] = subjects["edits"] / subjects["edits"].sum()
    subjects["angle"] = subjects["percentage"] * 360
    subjects["endAngle"] = subjects["angle"].cumsum()
    
    result = []
    
    result.append('<div style="display: flex; flex-flow: row wrap; align-items: center; justify-content: flex-start;">')
    result.append('<svg style="width: 50%; min-width: 200px; max-width: 25%;" viewBox="0 0 200 200">')
    
    for subject, angle, endAngle, color in zip(subjects.index, subjects["angle"], subjects["endAngle"], colors):
        result.append(circle(100, 100, 80, endAngle-angle, endAngle-0.00001, color))
    
    result.append('</svg>')
    result.append('<div style="width: 50%; padding: 1em;">')
    
    for subject, edits, color in zip(subjects.index, subjects["edits"], colors):
        result.append('<div style="display: flex; margin-bottom: 1em; align-items: center;">')
        result.append("""
            <div style="background-color: %s; width: 50px; height: 20px;
                         display: inline-block; border: 1px solid black;
                         margin-right: 0.5em;"></div>
        """ % color)
        result.append("<div>%s (%s Bearbeitungen)</div>" % (subject, edits))
        result.append('</div>')
    
    result.append('</div>')
    result.append('</div>')
    
    return "\n".join(result)

display(HTML(createSubjectChart(getSubjects(10))))

# Aktivitätsdiagramm

In [156]:
import numpy as np
from IPython.display import Markdown

def getPathFromPoints(points, progress, color, width):
    d = "M%s %s " % points[-1]
    
    for i in range(len(points)):
        point_progress = min(progress * len(points) - i, 1)
        
        if point_progress >= 0:
            next_point = (
                points[i-1][0] + point_progress * (points[i][0]-points[i-1][0]),
                points[i-1][1] + point_progress * (points[i][1]-points[i-1][1])
            )
                
            d += "L%s %s" % next_point
    
    return """
        <path d="%s" fill="none"
              stroke-linejoin="round"
              stroke-linecap="round"
              stroke="%s" stroke-width="%s" />
    """ % (d, color, width)

def hexagon(x, y, radius, color, progress):
    angles = np.linspace(0, 360, 7) + 360/6 * 3 + 30
    points = [polarToCartesian(x, y, radius, angle) for angle in angles]

    return getPathFromPoints(points, progress, color, 10)

def createProgressBars(bar_func, nr_actions, color):
    if nr_actions == 0:
        return ""
    
    log_actions = math.log10(nr_actions)
    paths = [bar_func(x=100, y=100, radius=20+20*i,
                      progress=min(1, log_actions - i), color=color)
             for i in range(math.ceil(log_actions))]
    
    return "\n".join(paths)

def createActivityChart(bar_func, nr_actions, kind, color,
                        max_edits=10000, color_missing_actions="#eee"):
    nr_actions = min(max_edits, nr_actions)
    
    result = '<div style="min-width: 200px; width: 25%;">'
    result += '<svg style="width: 100%;" viewBox="0 0 200 200">'
    result += createProgressBars(bar_func, max_edits, color_missing_actions)
    result += createProgressBars(bar_func, nr_actions, color)
    result += '</svg>'    
    result += '<p style="width: 100%%; text-align: center;"><b>%s %s</b></p>' % (nr_actions, kind)
    result += '</div>'
    
    return result

def eventType(id):
    if id in [1,2, 12, 17]:
        return "Themenbaum"
    if id in [4, 5]:
        return "Bearbeitungen"
    if id in [8, 9, 14]:
        return "Kommentare"
    if id in [6,11]:
        return "Reviewing"

def getActivities(userId):
    df = pd.read_sql("""
        select event_log.event_id from event_log
            where event_log.actor_id = %s and
            event_log.instance_id = 1;
    """ % userId, db)
    df["event"] = df["event_id"].map(eventType).dropna()
    df.sort_index(inplace=True)
    
    return df.groupby("event").count().rename(columns={"event_id": "actions"})

def getActivityCharts(activities):
    result = []
    
    result.append('<div style="display: flex; flex-flow: row wrap; width: 100%; justify-content: space-around;">')
    for kind, actions, color in zip(activities.index, activities["actions"], colors):
        result.append(createActivityChart(hexagon, actions, kind, color))
    
    result.append('</div>')
    
    return "\n".join(result)

display(HTML(getActivityCharts(getActivities(10))))

In [157]:
def getUsername(userId):
    return querySingleton("""
        select username from user where id = %s;
    """ % userId)[0]

def showExample(userId):
    username = getUsername(userId)
    display(Markdown("## %s" % username))
    display(Markdown("### %ss Aktivitäten nach Fach" % username))
    display(HTML(createSubjectChart(getSubjects(userId))))
    display(Markdown("### %ss Aktivitäten nach Bereich" % username))
    display(HTML(getActivityCharts(getActivities(userId))))

for userId in [ 266, 18981, 73435, 26217, 102817,
           146443, 146133, 69513, 324, 53589,
           63374, 27693, 92258, 185650, 146880]:
    showExample(userId)

## Renate

### Renates Aktivitäten nach Fach

### Renates Aktivitäten nach Bereich

## wolfgang

### wolfgangs Aktivitäten nach Fach

...

### Ilarias Aktivitäten nach Fach

### Ilarias Aktivitäten nach Bereich

## Larisa_J

### Larisa_Js Aktivitäten nach Fach

### Larisa_Js Aktivitäten nach Bereich